In [1]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import time

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
soup = BeautifulSoup(data.text)
standings_table = soup.select("table.stats_table")[0]

In [5]:
links = standings_table.find_all("a")
links = [l.get("href") for l in links]
links = [l for l in links if "/squads" in l]

In [6]:
team_urls = [f"https://fbref.com/{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
matches = pd.read_html(data.text, match="Scores & Fixtures ")[0]

/var/folders/z5/cx4ddsg557q0lqzd5939yrkh0000gn/T/ipykernel_10143/300854440.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures ")[0]


In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

/var/folders/z5/cx4ddsg557q0lqzd5939yrkh0000gn/T/ipykernel_10143/2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [12]:
shooting.columns = shooting.columns.droplevel()

In [13]:
team_data = matches.merge(shooting[["Date", "Sh","SoT", "Dist", "FK", "PK", "PKatt"]])

In [14]:
years = list(range(2024,2017, -1))
# list containing different dataframes: all matches for each team in each season
all_matches = []

In [15]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [16]:
# looping through all the years to take out all the match data for the different teams
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    #getting the data for previous seasons, getting the link and scrapping the data from there
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        #since I only want the Premier league data, we filter it here
        team_data = team_data[team_data["Comp"] == "Premier League"]
        #create the columns since they are not already in the table, for season and team
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        #adding this in order to scrape slower in order not to get blocked from the website
        time.sleep(3)

/var/folders/z5/cx4ddsg557q0lqzd5939yrkh0000gn/T/ipykernel_10143/3864778499.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/z5/cx4ddsg557q0lqzd5939yrkh0000gn/T/ipykernel_10143/3864778499.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
/var/folders/z5/cx4ddsg557q0lqzd5939yrkh0000gn/T/ipykernel_10143/3864778499.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/z5/cx4ddsg557q0lqzd5939yrkh0000gn/T/ipyke

In [17]:
all_matches_df = pd.concat(all_matches)
all_matches_df.columns = [c.lower() for c in all_matches_df.columns]
all_matches_df


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,2024,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,2024,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,2024,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,2024,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,29.0,13.0,16.4,1.0,0,0,2024,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2018-04-15,16:00,Premier League,Matchweek 34,Sun,Away,W,1,0,Manchester Utd,...,Match Report,NaN,10.0,4.0,18.1,0.0,0,0,2018,West Bromwich Albion
39,2018-04-21,12:30,Premier League,Matchweek 35,Sat,Home,D,2,2,Liverpool,...,Match Report,NaN,13.0,6.0,17.7,0.0,0,0,2018,West Bromwich Albion
40,2018-04-28,15:00,Premier League,Matchweek 36,Sat,Away,W,1,0,Newcastle Utd,...,Match Report,NaN,9.0,2.0,20.1,0.0,0,0,2018,West Bromwich Albion
41,2018-05-05,15:00,Premier League,Matchweek 37,Sat,Home,W,1,0,Tottenham,...,Match Report,NaN,9.0,1.0,10.2,0.0,0,0,2018,West Bromwich Albion


In [18]:
all_matches_df["team"].value_counts()

team
Manchester City             266
Manchester United           266
Arsenal                     266
Everton                     266
Crystal Palace              266
West Ham United             266
Brighton and Hove Albion    266
Newcastle United            266
Chelsea                     266
Tottenham Hotspur           266
Liverpool                   266
Leicester City              228
Wolverhampton Wanderers     228
Southampton                 228
Burnley                     228
Bournemouth                 190
Aston Villa                 190
Fulham                      152
Watford                     152
Leeds United                114
Brentford                   114
Sheffield United            114
Nottingham Forest            76
Norwich City                 76
West Bromwich Albion         76
Huddersfield Town            76
Luton Town                   38
Cardiff City                 38
Swansea City                 38
Stoke City                   38
Name: count, dtype: int64

In [19]:
all_matches_df["season"].value_counts()

season
2024    760
2023    760
2022    760
2021    760
2020    760
2019    760
2018    760
Name: count, dtype: int64

In [20]:
all_matches_df["round"].value_counts()

round
Matchweek 1     140
Matchweek 30    140
Matchweek 23    140
Matchweek 24    140
Matchweek 25    140
Matchweek 18    140
Matchweek 26    140
Matchweek 27    140
Matchweek 28    140
Matchweek 31    140
Matchweek 2     140
Matchweek 32    140
Matchweek 33    140
Matchweek 29    140
Matchweek 35    140
Matchweek 36    140
Matchweek 37    140
Matchweek 34    140
Matchweek 22    140
Matchweek 21    140
Matchweek 20    140
Matchweek 19    140
Matchweek 3     140
Matchweek 4     140
Matchweek 5     140
Matchweek 6     140
Matchweek 7     140
Matchweek 8     140
Matchweek 9     140
Matchweek 10    140
Matchweek 11    140
Matchweek 12    140
Matchweek 13    140
Matchweek 14    140
Matchweek 15    140
Matchweek 16    140
Matchweek 17    140
Matchweek 38    140
Name: count, dtype: int64

In [21]:
all_matches_df.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team'],
      dtype='object')

In [22]:
#drop non relevant columns
all_matches_df = all_matches_df.drop(columns=['notes', 'attendance', 'match report'])

# Rename columns
all_matches_df = all_matches_df.rename(columns={'round': 'Matchweek', 'team': 'HomeTeam', 'opponent': 'AwayTeam'})

# Extract the number from the 'Matchweek' column
all_matches_df['Matchweek'] = all_matches_df['Matchweek'].str.extract(r'(\d+)').astype(int)
all_matches_df

,date,time,comp,Matchweek,day,venue,result,gf,ga,AwayTeam,...,formation,referee,sh,sot,dist,fk,pk,pkatt,season,HomeTeam
1,2023-08-11,20:00,Premier League,1,Fri,Away,W,3,0,Burnley,...,4-2-3-1,Craig Pawson,17.0,8.0,13.9,0.0,0,0,2024,Manchester City
3,2023-08-19,20:00,Premier League,2,Sat,Home,W,1,0,Newcastle Utd,...,4-2-3-1,Robert Jones,14.0,4.0,17.9,0.0,0,0,2024,Manchester City
4,2023-08-27,14:00,Premier League,3,Sun,Away,W,2,1,Sheffield Utd,...,4-2-3-1,Jarred Gillett,29.0,9.0,17.3,2.0,0,1,2024,Manchester City
5,2023-09-02,15:00,Premier League,4,Sat,Home,W,5,1,Fulham,...,4-2-3-1,Michael Oliver,6.0,4.0,14.8,0.0,1,1,2024,Manchester City
6,2023-09-16,15:00,Premier League,5,Sat,Away,W,3,1,West Ham,...,4-2-3-1,Andy Madley,29.0,13.0,16.4,1.0,0,0,2024,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2018-04-15,16:00,Premier League,34,Sun,Away,W,1,0,Manchester Utd,...,4-4-1-1,Paul Tierney,10.0,4.0,18.1,0.0,0,0,2018,West Bromwich Albion
39,2018-04-21,12:30,Premier League,35,Sat,Home,D,2,2,Liverpool,...,4-4-1-1,Stuart Attwell,13.0,6.0,17.7,0.0,0,0,2018,West Bromwich Albion
40,2018-04-28,15:00,Premier League,36,Sat,Away,W,1,0,Newcastle Utd,...,4-4-1-1,David Coote,9.0,2.0,20.1,0.0,0,0,2018,West Bromwich Albion
41,2018-05-05,15:00,Premier League,37,Sat,Home,W,1,0,Tottenham,...,4-4-1-1,Mike Jones,9.0,1.0,10.2,0.0,0,0,2018,West Bromwich Albion


In [23]:
# Create a mapping dictionary from df
team_name_mapping = {
    'Bournemouth': 'AFC Bournemouth',
    'Arsenal': 'Arsenal FC',
    'Aston Villa': 'Aston Villa',
    'Brentford': 'Brentford FC',
    'Brighton': 'Brighton & Hove Albion',
    'Burnley': 'Burnley FC',
    'Cardiff': 'Cardiff City',
    'Chelsea': 'Chelsea FC',
    'Crystal Palace': 'Crystal Palace',
    'Everton': 'Everton FC',
    'Fulham': 'Fulham FC',
    'Huddersfield': 'Huddersfield Town',
    'Hull': 'Hull City',
    'Leeds': 'Leeds United',
    'Leicester': 'Leicester City',
    'Liverpool': 'Liverpool FC',
    'Luton': 'Luton Town',
    'Man City': 'Manchester City',
    'Man United': 'Manchester United',
    'Middlesbrough': 'Middlesbrough FC',
    'Newcastle': 'Newcastle United',
    'Norwich': 'Norwich City',
    'Nott\'m Forest': 'Nottingham Forest',
    'QPR': 'Queens Park Rangers',
    'Sheffield United': 'Sheffield United',
    'Southampton': 'Southampton FC',
    'Stoke': 'Stoke City',
    'Sunderland': 'Sunderland AFC',
    'Swansea': 'Swansea City',
    'Tottenham': 'Tottenham Hotspur',
    'Watford': 'Watford FC',
    'West Brom': 'West Bromwich Albion',
    'West Ham': 'West Ham United',
    'Wolves': 'Wolverhampton Wanderers'
}

# Check if the 'HomeTeam' and 'AwayTeam' columns exist and replace values
if 'HomeTeam' in all_matches_df.columns:
    all_matches_df['HomeTeam'] = all_matches_df['HomeTeam'].replace(team_name_mapping)
    
if 'AwayTeam' in all_matches_df.columns:
    all_matches_df['AwayTeam'] = all_matches_df['AwayTeam'].replace(team_name_mapping)

# Verify the replacements
print(all_matches_df['HomeTeam'].value_counts().sort_index())
print(all_matches_df['AwayTeam'].value_counts().sort_index())

HomeTeam
AFC Bournemouth             190
Arsenal FC                  266
Aston Villa                 190
Brentford FC                114
Brighton and Hove Albion    266
Burnley FC                  228
Cardiff City                 38
Chelsea FC                  266
Crystal Palace              266
Everton FC                  266
Fulham FC                   152
Huddersfield Town            76
Leeds United                114
Leicester City              228
Liverpool FC                266
Luton Town                   38
Manchester City             266
Manchester United           266
Newcastle United            266
Norwich City                 76
Nottingham Forest            76
Sheffield United            114
Southampton FC              228
Stoke City                   38
Swansea City                 38
Tottenham Hotspur           266
Watford FC                  152
West Bromwich Albion         76
West Ham United             266
Wolverhampton Wanderers     228
Name: count, dtype: int64
AwayT

In [24]:
all_matches_df.tail(50)

,date,time,comp,Matchweek,day,venue,result,gf,ga,AwayTeam,...,formation,referee,sh,sot,dist,fk,pk,pkatt,season,HomeTeam
29,2018-02-10,15:00,Premier League,27,Sat,Home,D,1,1,Brighton & Hove Albion,...,4-3-3,Robert Madley,14.0,1.0,21.8,0.0,0,1,2018,Stoke City
30,2018-02-24,12:30,Premier League,28,Sat,Away,D,1,1,Leicester City,...,4-1-4-1,Michael Oliver,6.0,2.0,29.8,1.0,0,0,2018,Stoke City
31,2018-03-03,15:00,Premier League,29,Sat,Away,D,0,0,Southampton FC,...,4-1-4-1,Anthony Taylor,12.0,3.0,23.7,0.0,0,0,2018,Stoke City
32,2018-03-12,20:00,Premier League,30,Mon,Home,L,0,2,Manchester City,...,4-3-3,Jonathan Moss,3.0,0.0,19.3,0.0,0,0,2018,Stoke City
33,2018-03-17,15:00,Premier League,31,Sat,Home,L,1,2,Everton FC,...,4-3-3,Martin Atkinson,9.0,4.0,19.2,0.0,0,0,2018,Stoke City
34,2018-04-01,13:30,Premier League,32,Sun,Away,L,0,3,Arsenal FC,...,4-4-1-1,Craig Pawson,8.0,2.0,17.5,0.0,0,0,2018,Stoke City
35,2018-04-07,15:00,Premier League,33,Sat,Home,L,1,2,Tottenham Hotspur,...,4-4-1-1,Graham Scott,10.0,4.0,15.4,1.0,0,0,2018,Stoke City
36,2018-04-16,20:00,Premier League,34,Mon,Away,D,1,1,West Ham United,...,4-4-1-1,Michael Oliver,11.0,6.0,15.3,0.0,0,0,2018,Stoke City
37,2018-04-22,13:30,Premier League,35,Sun,Home,D,1,1,Burnley FC,...,4-4-2,Mike Dean,8.0,3.0,20.7,0.0,0,0,2018,Stoke City
38,2018-04-28,12:30,Premier League,36,Sat,Away,D,0,0,Liverpool FC,...,4-4-2,Andre Marriner,5.0,1.0,9.9,0.0,0,0,2018,Stoke City


In [25]:
all_matches_df["season"].value_counts()

season
2024    760
2023    760
2022    760
2021    760
2020    760
2019    760
2018    760
Name: count, dtype: int64

In [26]:
all_matches_df.columns

Index(['date', 'time', 'comp', 'Matchweek', 'day', 'venue', 'result', 'gf',
       'ga', 'AwayTeam', 'xg', 'xga', 'poss', 'captain', 'formation',
       'referee', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt', 'season',
       'HomeTeam'],
      dtype='object')

In [37]:
all_matches_df.rename(columns={'date': 'Date'}, inplace=True)
all_matches_df["Date"] = pd.to_datetime(all_matches_df["Date"])


all_matches_df["Date"]

1    2023-08-11
3    2023-08-19
4    2023-08-27
5    2023-09-02
6    2023-09-16
        ...    
38   2018-04-15
39   2018-04-21
40   2018-04-28
41   2018-05-05
42   2018-05-13
Name: Date, Length: 5320, dtype: datetime64[ns]

In [38]:
all_matches_df.dtypes

Date         datetime64[ns]
time                 object
comp                 object
Matchweek             int64
day                  object
venue                object
result               object
gf                   object
ga                   object
AwayTeam             object
xg                  float64
xga                 float64
poss                float64
captain              object
formation            object
referee              object
sh                  float64
sot                 float64
dist                float64
fk                  float64
pk                    int64
pkatt                 int64
season                int64
HomeTeam             object
dtype: object

In [34]:
all_matches_df.to_csv('all_matches_matchweek.csv', index=False)

In [42]:
all_matches_df.isna().sum()

Date         0
time         0
comp         0
Matchweek    0
day          0
venue        0
result       0
gf           0
ga           0
AwayTeam     0
xg           0
xga          0
poss         0
captain      0
formation    0
referee      0
sh           0
sot          0
dist         3
fk           0
pk           0
pkatt        0
season       0
HomeTeam     0
dtype: int64

In [43]:
all_matches_df["season"].value_counts()

season
2024    760
2023    760
2022    760
2021    760
2020    760
2019    760
2018    760
Name: count, dtype: int64